# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [15]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [3]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [23]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_train_copia = X_train.copy()
    X_val_copia = X_val.copy()
    
    if hyperparams.get('use_pca'):
        # ...
        if hyperparams.get('pca_standardize'):
            scaler_pca = StandardScaler()
            X_train_copia = scaler_pca.fit_transform(X_train_copia)
            X_val_copia = scaler_pca.transform(X_val_copia)
        componenti = hyperparams['pca_components'];
        pca = PCA(n_components = componenti)
        X_train_copia = pca.fit_transform(X_train_copia)
        X_val = pca.transform(X_val_copia)

    if hyperparams.get('data_standardize'):
        # ...
        scaler = StandardScaler()
        X_train_copia = scaler.fit_transform(X_train_copia)
        X_val_copia = scaler.transform(X_val_copia)

    # Aggiunge il termine costante ai dati
    # ...
    X_train_copia = np.c_[np.ones(X_train_copia.shape[0]), X_train_copia]
    X_val_copia = np.c_[np.ones(X_val_copia.shape[0]), X_val_copia]

    # Calcolo della soluzione di regressione lineare
    if hyperparams.get('use_regularization'):
        # ...
        X_trasposta = X_train_copia.T @ X_train_copia
        lmb = hyperparams.get('reg_lambda', 0.0)
        matr_identita = np.eye(X_train_copia.shape[1])
        matr_identita[0, 0] = 0
        peso = np.linalg.inv(X_trasposta + lmb * matr_identita) @ X_train_copia.T @ y_train
    else:
        # ...
        peso = np.linalg.inv(X_train_copia.T @ X_train_copia) @ X_train_copia.T @ y_train

    # Calcolo predizioni
    # ...
    y_pred = X_val_copia @ peso

    # Calcola il MAE
    # ...
    mae = mean_absolute_error(y_val, y_pred)

    return mae, peso

# Dividi il dataset in training e test set
# ...
X, y = shuffle(X, y, random_state=42)
n_total = len(X)
n_train = int(0.6 * n_total)
n_val = int(0.2 * n_total)
n_test = n_total - n_train - n_val

X_train = X.iloc[:n_train]
y_train = y.iloc[:n_train]

X_val = X.iloc[n_train:n_train + n_val]
y_val = y.iloc[n_train:n_train + n_val]

X_test = X.iloc[n_train + n_val:]
y_test = y.iloc[n_train + n_val:]

# Dividi il training set in training set effettivo e validation set
# ...
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

# Trova la configurazione di iperparametri migliore
# ...
best_mae = float('inf')
best_config = None
best_peso = None

for config in configs:
    mae, peso = pipeline(X_train, y_train, X_val, y_val, config)
    if mae < best_mae:
        best_mae = mae
        best_config = config
        best_peso = peso

print("Configurazione migliore possibile: ", best_config)
print("MAE relativa a validation set: ", best_mae)

# Riallena il modello sul training set completo
# ...
X_trainval_copia = X_trainval.copy()
X_test_copia = X_test.copy()

if best_config.get('use_pca'):
    if best_config.get('pca_standardize'):
        scaler_pca = StandardScaler()
        X_trainval_copia = scaler_pca.fit_transform(X_trainval_copia)
        X_test_copia = scaler_pca.transform(X_test_copia)
        
    pca = PCA(n_components=best_config['pca_components'])
    X_trainval_copia = pca.fit_transform(X_trainval_copia)
    X_test_copia = pca.transform(X_test_copia)

if best_config.get('data_standardize'):
    scaler = StandardScaler()
    X_trainval_copia = scaler.fit_transform(X_trainval_copia)
    X_test_copia = scaler.transform(X_test_copia)

X_trainval_comp = np.c_[np.ones(X_trainval_copia.shape[0]), X_trainval_copia]
X_test_comp = np.c_[np.ones(X_test_copia.shape[0]), X_test_copia]

if best_config.get('use_regularization'):
    lmb = best_config.get('reg_lambda', 0.0)
    matr_identita = np.eye(X_trainval_comp.shape[1])
    matr_identita[0,0] = 0
    peso_comp = np.linalg.inv(X_trainval_comp.T @ X_trainval_comp + lmb * matr_identita) @ X_trainval_comp.T @ y_trainval
else:
    peso_comp = np.linalg.inv(X_trainval_comp.T @ X_trainval_comp) @ X_trainval_comp.T @ y_trainval
# Calcola il MAE sul test set
# ...
y_pred_test = X_test_comp @ peso_comp
mae_test = mean_absolute_error(y_test, y_pred_test)

# Stampa  risultati
# ...
print("Configurazione migliore possibile: ", best_config)
print("MAE relativa a validation set: ", best_mae)
print("MAE relativa a test set: ", mae_test)

Configurazione migliore possibile:  {'data_standardize': True, 'use_regularization': True, 'lambda': 0.1}
MAE relativa a validation set:  0.5830561814064897
Configurazione migliore possibile:  {'data_standardize': True, 'use_regularization': True, 'lambda': 0.1}
MAE relativa a validation set:  0.5830561814064897
MAE relativa a test set:  0.5960317536180987


/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
